# Polars 入门

![Polars](https://raw.githubusercontent.com/pola-rs/polars-static/master/banner/polars_github_banner.svg)

`Polars` 是一个用于数据操作的 Rust 库，它提供了类似于 `Pandas` 的 API，但是比 `Pandas` `更快。Polars` 也提供了其他运行时的包，可以在 Python/NodeJS 中使用 `Polars`。

在这个笔记本中，我们将主要学习如何在 Python 中使用 `Polars`。NodeJS 接口基本一致，可以

## 0. 安装

### Python (via PyPI)

```bash
pip install polars
```

### NodeJS (via npm)

```bash
npm install nodejs-polars
```


In [1]:
import polars as pl
import pandas as pd
import dataclasses
import ipywidgets as widgets
import numpy as np

np.random.seed(46)

## 1. 初始化构造

`Polars` 构造函数的入参中我们需要关注的一般只有两个，`data` 和 `schema`，可以理解为 `Pandas` 构造函数中的 `data` 和 `dtypes`。

其中 `data` 支持 `dict`, `Sequence`, `ndarray`, `Series`, or `pandas.DataFrame`。与 `Pandas` 不同的是不支持从 `records`/`tuple` 构建的方式。


In [2]:
@dataclasses.dataclass
class Record:
    x: float
    y: int
    z: str


pd.DataFrame([Record(x=0.1, y=1, z="hello"), Record(x=0.2, y=2, z="pandas")])

,x,y,z
0,0.1,1,hello
1,0.2,2,pandas


详细的构造函数签名可以参考 `DataFrame.__init__`


In [3]:
print(pl.DataFrame.__doc__)


    Two-dimensional data structure representing data as a table with rows and columns.

    Parameters
    ----------
    data : dict, Sequence, ndarray, Series, or pandas.DataFrame
        Two-dimensional data in various forms; dict input must contain Sequences,
        Generators, or a `range`. Sequence may contain Series or other Sequences.
    schema : Sequence of str, (str,DataType) pairs, or a {str:DataType,} dict
        The schema of the resulting DataFrame. The schema may be declared in several
        ways:

        * As a dict of {name:type} pairs; if type is None, it will be auto-inferred.
        * As a list of column names; in this case types are automatically inferred.
        * As a list of (name,type) pairs; this is equivalent to the dictionary form.

        If you supply a list of column names that does not match the names in the
        underlying data, the names given here will overwrite them. The number
        of names given in the schema should match the underl

In [4]:
# Simple
pl.DataFrame(
    {
        "x": [1, 2, 3],
        "y": None,
        "z": ["female", "male", "female"],
    }
)

x,y,z
i64,null,str
1,null,"""female"""
2,null,"""male"""
3,null,"""female"""


`schema` 支持 `list[str]` / `list[tuple[str, pl.DataTypeClass]]` / `dict[str, pl.DataTypeClass]`，不支持从 `str` 构建类型

> 某些情况下我们必须要从 `str` 构建类型的话可以采用 `polars` 提供的工具函数 `pl.datatypes.convert.dtype_to_ffiname`，具体见 [工具函数](#PolarsUtils)

需要注意的是 `pl.Categorical` 和 `pl.Enum` 这两个类型，类似于 `pd.Categorical`，这两个类型都是代表分类变量的，但是区别在于

1. `polars` 强制要求 `Categorical`/`Enum` 类型先是字符串类型，否则 `raise`
2. Categorical 和 Enum 存在区别，`Categorical` 在构建时可以不指定分类级别，然而 Enum 必须要指定
3. Categorical 支持通过字符序/出现先后顺序指定大小，Enum 通过指定的级别的索引指定大小


In [5]:
print("pl.Categorical")
print("================")
print(pl.Categorical.__doc__)

print("pl.Enum")
print("================")
print(pl.Enum.__doc__)

pl.Categorical

    A categorical encoding of a set of strings.

    Parameters
    ----------
    ordering : {'lexical', 'physical'}
        Ordering by order of appearance (`'physical'`, default)
        or string value (`'lexical'`).
    
pl.Enum

    A fixed set categorical encoding of a set of strings.

    .. warning::
        This functionality is considered **unstable**.
        It is a work-in-progress feature and may not always work as expected.
        It may be changed at any point without it being considered a breaking change.

    Parameters
    ----------
    categories
        The categories in the dataset. Categories must be strings.
    


In [6]:
simple = pl.DataFrame(
    {
        "SEX": ["F", "M", "M", "F"],
        "ID": [1, 2, 2, 3],
        "WEIGHT": [70.2, 88.8, 49.3, 55],
        "GROUP": ["B", "A", "A", "B"],
    },
    {
        "SEX": pl.Categorical,
        "ID": pl.Int8,
        "WEIGHT": float,
        "GROUP": pl.Enum(["A", "B"]),
    },
)
simple_pandas = simple.to_pandas()
simple


SEX,ID,WEIGHT,GROUP
cat,i8,f64,enum
"""F""",1,70.2,"""B"""
"""M""",2,88.8,"""A"""
"""M""",2,49.3,"""A"""
"""F""",3,55.0,"""B"""


## 2. 构成元素

Polars 中的元素主要有 `DataFrame`， `Series`，`Expr`, 其中 `DataFrame` 和 `Series` 与 `Pandas` 中的 `DataFrame` 和 `Series` 类似，`Expr` 是 Polars 中的表达式，能够表示一些筛选/聚合的条件。与 pandas 不同的是 `Expr` 是 `lazy` 的，只有在需要的时候才会计算，而 pandas 是当场执行生成 boolean masked series/dataframe。对于巨量数据，pandas 的劣势相当明显


In [7]:
N = 1_000_000
maga_data = pl.DataFrame(
    {
        "SEX": np.random.choice(["F", "M"], N),
        "WEIGHT": np.random.normal(60, 10, N),
    }
)
maga_data_pandas = maga_data.to_pandas()

In [8]:
%%timeit -n 10
(pl.col("SEX") == "M").and_(pl.col("WEIGHT") > pl.col("WEIGHT").mean())

6.29 μs ± 1.77 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit -n 10
maga_data.filter(
    (pl.col("SEX") == "M").and_(pl.col("WEIGHT") > pl.col("WEIGHT").mean())
)

1.92 ms ± 106 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit -n 10
(maga_data_pandas["SEX"] == "M") & (
    maga_data_pandas["WEIGHT"] > maga_data_pandas["WEIGHT"].mean()
)

37.3 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


polars 的 `Expr` 非常的强大且语义性强，可以通过 `Expr` 构建复杂的筛选/聚合条件，而不需要使用 `lambda` 函数。多个条件可以通过 `and_(&)` 和 `or_(|)` 链接，同时也支持一系列的统计量计算，如 `sum`, `mean`, `std`, `var`, 等。


In [11]:
widgets.Select(
    options=filter(lambda v: not v.startswith("_"), pl.col("SEX").__dir__()),
    disabled=False,
    rows=15,
)

Select(options=('deserialize', 'to_physical', 'any', 'all', 'arg_true', 'sqrt', 'cbrt', 'log10', 'exp', 'alias…

In [12]:
# 体重大于 25% 分位数的男性
(pl.col("WEIGHT") > pl.col("WEIGHT").quantile(0.25)).and_(pl.col("SEX") == "M")

<Expr ['[([(col("WEIGHT")) > (col("WEI…'] at 0x10E853310>

In [13]:
# 年龄小于中位数，年收入和副业收入的总和大于 90% 分位数，要么有留学经历但未婚，要么没有留学经历但已婚
(
    (pl.col("AGE") < pl.col("AGE").median())
    & (
        pl.col("INCOME") + pl.col("SIDE_INCOME")
        > pl.col("INCOME").add(pl.col("SIDE_INCOME")).quantile(0.9)
    )
    & (
        ((pl.col("STUDY_ABROAD") == "Y") & (pl.col("MARRIED") == "N"))
        | ((pl.col("STUDY_ABROAD") == "N") & (pl.col("MARRIED") == "Y"))
    )
)

<Expr ['[([([(col("AGE")) < (col("AGE"…'] at 0x10E8527A0>

## 3. 数据处理

`polars` 和 `pandas` 最大的区别（或者说我最关心的差异）是 `polars` 的所有数据处理都会返回一份拷贝，而 `pandas` 会返回视图。这样的设计使得 `polars` 更加安全，同时由于 Rust 原生的 Ownership 机制，使得 `polars` 的性能并不会收到太大影响。同时这样的特性可以有效的帮助开发者避免一些潜在的错误，特别是在一些存在数据处理，但是并不想影响原数据的情况下，如多线程环境，函数式编程，底层算法等。


In [14]:
example = {
    "a": [1, 2, 3, 4, 5],
    "b": [5.5, 4.4, 3.3, 2.2, 1.1],
    "c": ["a", "b", "c", "d", "e"],
}
example_pl = pl.DataFrame(example)
example_pd = pd.DataFrame(example)


In [15]:
def some_func(df: pl.DataFrame | pd.DataFrame) -> None:
    df["a"] = 123

In [16]:
some_func(example_pd)
example_pd  # Changed！很危险

,a,b,c
0,123,5.5,a
1,123,4.4,b
2,123,3.3,c
3,123,2.2,d
4,123,1.1,e


In [17]:
%%capture
# 会报错，赞 👍
some_func(example_pl)

TypeError: DataFrame object does not support `Series` assignment by index

Use `DataFrame.with_columns`.

使用 `polars` 数据处理和 R 的 `dplyr` 风格很相似，可以通过 `DataFrame` 的方法链式调用，或者通过 `pl` 模块提供的函数式编程接口。大部分的方法和 SQL 中的操作是一一对应的，如 `select`, `filter`, `groupby`, `join`, `sort`, `agg`, `pivot`, `explode` 等等。本文档会主要介绍一些常用的方法。具体更多的方法和函数接口可以参考 [Python API 文档](https://docs.pola.rs/api/python/stable/reference/dataframe/modify_select.html) 或者 [NodeJS API 文档](https://pola-rs.github.io/nodejs-polars/interfaces/pl.DataFrame-1.html)。


In [18]:
student_performance = pl.read_csv("data/StudentPerformance.csv")
student_performance.head(10)

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
1003,15,0,2,3,4.21057,26,0,2,0,0,0,0,0.112602,4.0
1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
1006,18,0,0,1,8.191219,0,0,1,1,0,0,0,3.084184,1.0
1007,15,0,1,1,15.60168,10,0,3,0,1,0,0,2.748237,2.0
1008,15,1,1,4,15.424496,22,1,1,1,0,0,0,1.360143,4.0
1009,17,0,0,0,4.562008,1,0,2,0,1,0,1,2.896819,2.0


### 3.1 类型 cast

`polars` 提供了 `cast` 方法用于类型转换，需要注意的是数值类型转 Categorical / Enum 类型时，需要先转为字符串类型。


In [19]:
%%capture
# 报错，Gender 原来是 i64，现在是 Categorical，需要先转 String
student_performance = student_performance.cast(
    {
        "Gender": pl.Categorical,
    }
)

ComputeError: cannot cast numeric types to 'Categorical'

In [20]:
student_performance = student_performance.cast({"Gender": pl.String}).cast(
    {"Gender": pl.Enum(["0", "1"])}
)
student_performance

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1001,17,"""1""",0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1002,18,"""0""",0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
1003,15,"""0""",2,3,4.21057,26,0,2,0,0,0,0,0.112602,4.0
1004,17,"""1""",0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
1005,17,"""1""",0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3388,18,"""1""",0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
3389,17,"""0""",0,1,7.583217,4,1,4,0,1,0,0,3.27915,4.0
3390,16,"""1""",0,2,6.8055,20,0,2,0,0,0,1,1.142333,2.0


`cast` 接口也支持类型的映射，如 `cast({pl.Float64: pl.String})`，这样可以一次性转换多列的类型。


In [21]:
student_performance.cast({pl.Float64: pl.String})

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,str,str
1001,17,"""1""",0,2,"""19.833722807854713""",7,1,2,0,0,1,0,"""2.929195591667681""","""2.0"""
1002,18,"""0""",0,1,"""15.40875605584674""",0,0,1,0,0,0,0,"""3.042914833436377""","""1.0"""
1003,15,"""0""",2,3,"""4.21056976881226""",26,0,2,0,0,0,0,"""0.1126022544661815""","""4.0"""
1004,17,"""1""",0,3,"""10.028829473958215""",14,0,3,1,0,0,0,"""2.0542181397029484""","""3.0"""
1005,17,"""1""",0,2,"""4.6724952729713305""",17,1,3,0,0,0,0,"""1.2880611817953875""","""4.0"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3388,18,"""1""",0,3,"""10.680554607694354""",2,0,4,1,0,0,0,"""3.4555094110343934""","""0.0"""
3389,17,"""0""",0,1,"""7.583217279598868""",4,1,4,0,1,0,0,"""3.2791497270250276""","""4.0"""
3390,16,"""1""",0,2,"""6.8054996446806175""",20,0,2,0,0,0,1,"""1.142332879526157""","""2.0"""


### 3.2 列选择

在数据处理中，我们经常需要选择一部分列进行处理，`polars` 提供了 `select` 方法来实现这个功能。当然可以直接通过 `operator[]` 来选择列


In [22]:
student_performance["Music"]

Music
i64
1
0
0
0
0
…
0
0
0


In [23]:
# 全选
student_performance.select(pl.all())

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1001,17,"""1""",0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1002,18,"""0""",0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
1003,15,"""0""",2,3,4.21057,26,0,2,0,0,0,0,0.112602,4.0
1004,17,"""1""",0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
1005,17,"""1""",0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3388,18,"""1""",0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
3389,17,"""0""",0,1,7.583217,4,1,4,0,1,0,0,3.27915,4.0
3390,16,"""1""",0,2,6.8055,20,0,2,0,0,0,1,1.142333,2.0


In [24]:
# 选中数值列
import polars.selectors as selectors

student_performance.select(selectors.numeric())

StudentID,Age,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1001,17,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1002,18,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
1003,15,2,3,4.21057,26,0,2,0,0,0,0,0.112602,4.0
1004,17,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
1005,17,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
3388,18,0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
3389,17,0,1,7.583217,4,1,4,0,1,0,0,3.27915,4.0
3390,16,0,2,6.8055,20,0,2,0,0,0,1,1.142333,2.0


In [25]:
# 选中所有以 A 或者 S 开头的列
student_performance.select(selectors.starts_with("A", "S"))

StudentID,Age,StudyTimeWeekly,Absences,Sports
i64,i64,f64,i64,i64
1001,17,19.833723,7,0
1002,18,15.408756,0,0
1003,15,4.21057,26,0
1004,17,10.028829,14,0
1005,17,4.672495,17,0
…,…,…,…,…
3388,18,10.680555,2,0
3389,17,7.583217,4,1
3390,16,6.8055,20,0


In [26]:
# 复杂选中
student_performance.select(
    "StudentID",
    selectors.by_dtype(pl.Float64, pl.Enum).and_(selectors.starts_with("G")),
    selectors.matches(r"^(A|B|C|D|E)").and_(selectors.numeric()) ** 2,
)

StudentID,Gender,GPA,GradeClass,Age,Ethnicity,Absences,Extracurricular
i64,enum,f64,f64,i64,i64,i64,i64
1001,"""1""",2.929196,2.0,289,0,49,0
1002,"""0""",3.042915,1.0,324,0,0,0
1003,"""0""",0.112602,4.0,225,4,676,0
1004,"""1""",2.054218,3.0,289,0,196,1
1005,"""1""",1.288061,4.0,289,0,289,0
…,…,…,…,…,…,…,…
3388,"""1""",3.455509,0.0,324,0,4,1
3389,"""0""",3.27915,4.0,289,0,16,0
3390,"""1""",1.142333,2.0,256,0,400,0


### 3.3 行选择

选择行的行为和选择列类似，`polars` 提供了 `filter` 方法来实现这个功能。同样也可以使用 `operator[]` 来选择行，和使用 `[]` 选择列的区别在于列选择时可以传入列名，而行选择时传入的是行索引（int）。由于 `polars.DataFrame` 默认没有行索引，或者说没有提供能够得到行索引的选择器，所以当我们期望使用行索引（index）作为值时，需要使用 `with_row_index` 方法来获取行索引。需要注意的是默认的行索引列名为 `index`，如果存在同名列，需要指定行索引列名。


In [27]:
indexed = student_performance.with_row_index()
indexed.tail(10)

index,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
u32,i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
2382,3383,16,"""0""",0,3,13.941823,20,0,2,0,0,0,0,1.191929,3.0
2383,3384,16,"""1""",2,2,11.736409,18,1,4,0,0,0,0,1.810038,3.0
2384,3385,15,"""1""",0,1,16.655581,13,1,3,0,1,0,0,2.539113,4.0
2385,3386,16,"""1""",0,1,1.445434,20,0,3,1,1,0,0,1.395631,1.0
2386,3387,16,"""0""",0,2,13.814021,14,0,2,0,0,0,0,2.000084,4.0
2387,3388,18,"""1""",0,3,10.680555,2,0,4,1,0,0,0,3.455509,0.0
2388,3389,17,"""0""",0,1,7.583217,4,1,4,0,1,0,0,3.27915,4.0
2389,3390,16,"""1""",0,2,6.8055,20,0,2,0,0,0,1,1.142333,2.0
2390,3391,16,"""1""",1,0,12.416653,17,0,2,0,1,1,0,1.803297,1.0


In [28]:
student_performance[np.random.choice(student_performance.height, 10, replace=True)]

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1464,15,"""0""",1,3,0.477684,3,0,3,0,1,0,0,2.75359,2.0
1027,16,"""0""",0,1,10.367993,2,0,2,0,1,0,0,2.948718,2.0
3250,15,"""0""",0,2,15.543718,28,0,3,1,0,1,0,1.262849,4.0
2022,15,"""1""",0,0,5.791654,2,0,1,1,0,0,0,2.815973,2.0
3262,18,"""0""",2,1,4.596521,12,1,2,1,0,0,0,2.376854,3.0
2580,17,"""1""",1,2,17.092349,18,0,0,0,1,0,0,1.377573,4.0
1924,17,"""0""",1,1,11.695136,15,0,4,0,0,1,0,2.3868,3.0
2317,16,"""0""",0,1,9.373033,0,0,2,0,1,1,1,3.287973,1.0
2137,17,"""1""",3,1,0.583413,1,0,2,0,1,1,0,3.204313,1.0


In [29]:
# 选中所有偶数行

student_performance[::2]

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1001,17,"""1""",0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1003,15,"""0""",2,3,4.21057,26,0,2,0,0,0,0,0.112602,4.0
1005,17,"""1""",0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
1007,15,"""0""",1,1,15.60168,10,0,3,0,1,0,0,2.748237,2.0
1009,17,"""0""",0,0,4.562008,1,0,2,0,1,0,1,2.896819,2.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3383,16,"""0""",0,3,13.941823,20,0,2,0,0,0,0,1.191929,3.0
3385,15,"""1""",0,1,16.655581,13,1,3,0,1,0,0,2.539113,4.0
3387,16,"""0""",0,2,13.814021,14,0,2,0,0,0,0,2.000084,4.0


In [30]:
# 或者使用帮助函数

student_performance.gather_every(2)

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1001,17,"""1""",0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1003,15,"""0""",2,3,4.21057,26,0,2,0,0,0,0,0.112602,4.0
1005,17,"""1""",0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0
1007,15,"""0""",1,1,15.60168,10,0,3,0,1,0,0,2.748237,2.0
1009,17,"""0""",0,0,4.562008,1,0,2,0,1,0,1,2.896819,2.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3383,16,"""0""",0,3,13.941823,20,0,2,0,0,0,0,1.191929,3.0
3385,15,"""1""",0,1,16.655581,13,1,3,0,1,0,0,2.539113,4.0
3387,16,"""0""",0,2,13.814021,14,0,2,0,0,0,0,2.000084,4.0


In [31]:
# 选中所有 %5=0 的行
indexed.filter(pl.col("index") % 5 == 0)

index,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
u32,i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
0,1001,17,"""1""",0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
5,1006,18,"""0""",0,1,8.191219,0,0,1,1,0,0,0,3.084184,1.0
10,1011,17,"""0""",0,1,11.851364,11,0,1,0,0,0,0,2.147172,3.0
15,1016,15,"""0""",0,2,9.728101,17,1,0,0,1,0,0,1.341521,4.0
20,1021,16,"""1""",0,3,2.621597,2,0,3,0,0,0,1,2.778411,2.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2370,3371,18,"""1""",2,1,2.912575,25,0,1,0,1,0,1,0.56942,4.0
2375,3376,18,"""0""",0,2,18.92529,24,0,1,1,0,1,1,1.164539,0.0
2380,3381,16,"""1""",0,2,2.138397,8,0,1,1,1,0,0,1.979379,1.0


In [32]:
# 选中 Age < 18， Gender = "0"，GPA < 2.0 的行
student_performance.filter(
    (pl.col("Age") < 18).and_(pl.col("Gender") == "0").and_(pl.col("GPA") < 2.0)
)

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1003,15,"""0""",2,3,4.21057,26,0,2,0,0,0,0,0.112602,4.0
1012,17,"""0""",0,1,7.598486,15,0,2,0,0,0,1,1.559595,4.0
1013,17,"""0""",1,1,10.038712,21,0,3,1,0,0,0,1.520078,4.0
1014,17,"""0""",1,2,12.101425,21,0,4,0,1,0,0,1.751581,4.0
1016,15,"""0""",0,2,9.728101,17,1,0,0,1,0,0,1.341521,4.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3341,16,"""0""",2,2,9.388697,18,0,1,0,1,0,0,1.288938,2.0
3347,17,"""0""",0,1,18.782174,19,0,2,0,0,1,0,1.823323,1.0
3357,16,"""0""",0,3,7.262652,22,0,3,0,0,0,0,0.766257,3.0


In [33]:
# 复杂选中
student_performance.filter(
    (pl.col("StudyTimeWeekly") < pl.col("StudyTimeWeekly").quantile(0.25)).or_(
        (pl.col("Absences") > pl.quantile("Absences", 0.75))
    ),
    (pl.col("ParentalEducation") >= 3).or_(pl.col("ParentalSupport") >= 3),
    pl.col("GPA") > 3.0,
)


StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1070,16,"""1""",2,1,0.941224,1,0,4,0,0,1,0,3.283917,1.0
1141,18,"""1""",0,0,0.785839,2,0,3,1,0,0,0,3.139482,1.0
1161,17,"""0""",2,2,0.99562,4,1,3,1,0,0,1,3.375994,1.0
1197,16,"""1""",0,4,1.989925,0,0,1,1,0,0,1,3.117354,1.0
1230,15,"""0""",2,2,4.336286,2,0,3,0,1,0,0,3.339094,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2633,18,"""1""",1,1,0.810183,3,0,3,0,1,1,0,3.133768,1.0
2738,17,"""0""",0,1,4.312795,4,0,3,1,1,0,0,3.152519,1.0
3149,16,"""1""",0,2,3.646704,6,0,3,1,1,1,0,3.183232,1.0


### 3.4 排序

`polars` 提供了 `sort` 方法来实现排序功能，和 `pandas` 类似，可以通过 `by` 参数指定排序的列，通过 `descending` 参数指定排序的方向。


In [34]:
student_performance.sort("GPA", descending=True).head(5)

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
1045,18,"""1""",0,1,18.921512,1,1,3,1,1,0,0,4.0,0.0
1443,15,"""0""",0,2,19.424398,0,0,3,0,1,1,1,4.0,0.0
2279,15,"""1""",0,2,18.899696,3,1,3,1,1,0,0,4.0,0.0
2706,18,"""1""",0,2,8.858282,0,1,4,1,0,0,0,4.0,0.0
2920,15,"""0""",3,1,17.442121,1,1,1,1,1,0,0,4.0,0.0


In [35]:
# 复杂排序
student_performance.sort(
    "GPA",
    "Age",
    pl.col("Sports") + pl.col("Music"),
    "StudyTimeWeekly",
    descending=[True, False, False, True],
).head(10)

StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
i64,i64,enum,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,f64,f64
2279,15,"""1""",0,2,18.899696,3,1,3,1,1,0,0,4.0,0.0
2920,15,"""0""",3,1,17.442121,1,1,1,1,1,0,0,4.0,0.0
1443,15,"""0""",0,2,19.424398,0,0,3,0,1,1,1,4.0,0.0
3029,16,"""1""",0,0,18.656924,0,1,4,1,0,0,1,4.0,0.0
3320,17,"""0""",0,2,9.285447,0,0,4,1,0,1,1,4.0,3.0
2706,18,"""1""",0,2,8.858282,0,1,4,1,0,0,0,4.0,0.0
1045,18,"""1""",0,1,18.921512,1,1,3,1,1,0,0,4.0,0.0
2261,17,"""1""",2,2,9.001905,2,1,3,1,1,0,0,3.984674,0.0
3355,16,"""1""",0,2,1.56549,4,1,4,1,1,0,0,3.979421,4.0


### 3.5 拼接

`polars` 可以通过 `concat`，`hstack`，`vstack`，`exend`，`join`，`merge_sorted`来实现拼接功能。

其中 `concat` 是 `polars` 的 namespace，其余都是 `DataFrame` 的方法。

需要特别关注

1. `extend` / `hstack` / `vstack` 支持 `inplace=True` 参数，默认为 `False`，如果 `inplace` 为 `True`，则会直接在原 DataFrame 上进行操作。
2. `extend` 默认不会从 `other` 中移动内存，而是尝试在当前内存大小中进行操作，所以如果不需要进行 resize 行为，`extend` 会更快。反之，如果当前申请的内存大小不足以容纳 `other`，则会进行 resize 操作后生成新的 chunk，影响 query 性能。所以如果是插入后查询，则倾向于使用 `extend`，如果是多次插入后查询，则倾向于使用 `vstack`。
3. `join` / `update` 签名基本一致，如 `on`, `left_on`, `right_on`, `how` 等，区别在于 `join` 是拼接行为，`update` 是更新行为。
4. `concat` 会要求所有的 DataFrame 的宽（`how="vertical"`）/高（`how="horizontal"`）度一致，否则会 `raise`。`hstack` 和 `vstack` 同理。


In [36]:
df = pl.DataFrame(
    {
        "foo": [1, 2, 3],
        "bar": [6.0, 7.0, 8.0],
        "ham": ["a", "b", "c"],
    }
)
other_df = pl.DataFrame(
    {
        "apple": ["x", "y", "z"],
        "ham": ["a", "b", "d"],
    }
)
df.join(other_df, on="ham")

foo,bar,ham,apple
i64,f64,str,str
1,6.0,"""a""","""x"""
2,7.0,"""b""","""y"""


In [37]:
df.join(other_df, on="ham", how="full")

foo,bar,ham,apple,ham_right
i64,f64,str,str,str
1,6.0,"""a""","""x""","""a"""
2,7.0,"""b""","""y""","""b"""
null,null,null,"""z""","""d"""
3,8.0,"""c""",null,null


In [38]:
df.join(other_df, on="ham", how="left", coalesce=False)

foo,bar,ham,apple,ham_right
i64,f64,str,str,str
1,6.0,"""a""","""x""","""a"""
2,7.0,"""b""","""y""","""b"""
3,8.0,"""c""",null,null


In [39]:
df.join(other_df, on="ham", how="left", coalesce=True)

foo,bar,ham,apple
i64,f64,str,str
1,6.0,"""a""","""x"""
2,7.0,"""b""","""y"""
3,8.0,"""c""",null


In [40]:
# 插入行
insert_at = 1
pl.concat(
    [
        df[:insert_at],
        pl.DataFrame({"foo": [4, 5], "bar": [9.0, 10.0], "ham": ["d", "e"]}),
        df[insert_at:],
    ]
)

foo,bar,ham
i64,f64,str
1,6.0,"""a"""
4,9.0,"""d"""
5,10.0,"""e"""
2,7.0,"""b"""
3,8.0,"""c"""


In [41]:
# 按须插入行
df.merge_sorted(
    pl.DataFrame({"foo": None, "bar": [6.5, 7.5, 8.5], "ham": "x"}), key="bar"
)


foo,bar,ham
i64,f64,str
1,6.0,"""a"""
null,6.5,"""x"""
2,7.0,"""b"""
null,7.5,"""x"""
3,8.0,"""c"""
null,8.5,"""x"""


### 3.6 添加列

`polars` 提供了 `with_columns` / `insert_column` 用于添加列，与 pandas 不同的是 `polars` 不支持直接通过 `[]` 添加列，并且函数行为会返回一个新的对象，而不是在原对象上进行操作。赋值的列支持

1. 表达式
2. Series
3. `a=b` 形式的 named pair


In [42]:
def int_to_roman(num: int) -> str:
    # Storing roman values of digits from 0-9
    # when placed at different places
    m = ["", "M", "MM", "MMM"]
    c = ["", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM "]
    x = ["", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC"]
    i = ["", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX"]

    # Converting to roman
    thousands = m[num // 1000]
    hundreds = c[(num % 1000) // 100]
    tens = x[(num % 100) // 10]
    ones = i[num % 10]

    ans = thousands + hundreds + tens + ones

    return ans

In [43]:
df.with_columns(
    (pl.col("foo") * 2).alias("foo * 2"),
    pl.Series("NullColumn", [None]),
    pl.Series("均匀分布", np.random.uniform(size=df.height)),
    bar_with_noise=pl.col("bar") + np.random.normal(0, 1, size=df.height),
).insert_column(
    0,
    pl.Series("NamedIndex", np.arange(df.height) + 1).map_elements(
        int_to_roman, return_dtype=pl.Categorical
    ),
)

NamedIndex,foo,bar,ham,foo * 2,NullColumn,均匀分布,bar_with_noise
str,i64,f64,str,i64,null,f64,f64
"""I""",1,6.0,"""a""",2,null,0.811194,8.088877
"""II""",2,7.0,"""b""",4,null,0.650119,6.013642
"""III""",3,8.0,"""c""",6,null,0.411437,8.191791


### 3.7 条件赋值

`pl.when()` / `pl.when().then()` / `pl.when().then().otherwise()` 用于条件赋值，类似于 SQL 中的 `CASE WHEN` 语句。在拼接数据/列时，经常会用到这个方法。


In [44]:
titanic = pl.read_csv("data/titanic.csv", infer_schema_length=None)
titanic

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
i64,i64,str,str,i64,i64,i64,str,f64,str,i64
1,3,"""Allison Hill""","""male""",17,4,2,"""43d75413-a939-4bd1-a516-b0d47d…",144.08,"""Q""",1
2,1,"""Noah Rhodes""","""male""",60,2,2,"""6334fa2a-8b4b-47e7-a451-5ae017…",249.04,"""S""",0
3,3,"""Angie Henderson""","""male""",64,0,0,"""61a66444-e2af-4629-9efb-336e2f…",50.31,"""Q""",1
4,3,"""Daniel Wagner""","""male""",35,4,0,"""0b6c03c8-721e-4419-afc3-e6495e…",235.2,"""C""",1
5,1,"""Cristian Santos""","""female""",70,0,3,"""436e3c49-770e-49db-b092-d40143…",160.17,"""C""",1
…,…,…,…,…,…,…,…,…,…,…
996,2,"""Brittany Ward""","""male""",69,2,3,"""75eb0e8f-a090-460d-a7c7-655a7b…",19.87,"""Q""",1
997,2,"""Edward Stanley""","""female""",61,4,4,"""e55b93ad-8976-4471-9d7f-57d1e4…",85.35,"""C""",0
998,3,"""Christina Johnson""","""male""",10,4,1,"""d06d3943-c8b3-415e-b664-704355…",444.22,"""C""",0


In [45]:
titanic1 = titanic.with_columns(
    pl.when(pl.col("Sex") == "male").then(0).otherwise(1).alias("SEXN"),
    pl.when(pl.col("Survived") == 1)
    .then(
        pl.when(pl.col("Age") < 18)
        .then(pl.lit("Survived Child"))
        .otherwise(pl.lit("Survived Adult"))
    )
    .otherwise(None)
    .alias("Survived Status"),
)
titanic1

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived,SEXN,Survived Status
i64,i64,str,str,i64,i64,i64,str,f64,str,i64,i32,str
1,3,"""Allison Hill""","""male""",17,4,2,"""43d75413-a939-4bd1-a516-b0d47d…",144.08,"""Q""",1,0,"""Survived Child"""
2,1,"""Noah Rhodes""","""male""",60,2,2,"""6334fa2a-8b4b-47e7-a451-5ae017…",249.04,"""S""",0,0,null
3,3,"""Angie Henderson""","""male""",64,0,0,"""61a66444-e2af-4629-9efb-336e2f…",50.31,"""Q""",1,0,"""Survived Adult"""
4,3,"""Daniel Wagner""","""male""",35,4,0,"""0b6c03c8-721e-4419-afc3-e6495e…",235.2,"""C""",1,0,"""Survived Adult"""
5,1,"""Cristian Santos""","""female""",70,0,3,"""436e3c49-770e-49db-b092-d40143…",160.17,"""C""",1,1,"""Survived Adult"""
…,…,…,…,…,…,…,…,…,…,…,…,…
996,2,"""Brittany Ward""","""male""",69,2,3,"""75eb0e8f-a090-460d-a7c7-655a7b…",19.87,"""Q""",1,0,"""Survived Adult"""
997,2,"""Edward Stanley""","""female""",61,4,4,"""e55b93ad-8976-4471-9d7f-57d1e4…",85.35,"""C""",0,1,null
998,3,"""Christina Johnson""","""male""",10,4,1,"""d06d3943-c8b3-415e-b664-704355…",444.22,"""C""",0,0,null


In [46]:
titanic1.group_by(
    pl.when(pl.col("Survived Status").is_not_null())
    .then(pl.col("Survived Status"))
    .otherwise(pl.lit("Died"))
    .alias("状态"),
    pl.col("Pclass").alias("舱位"),
).agg(
    (pl.col("SEXN") == 0).sum().alias("男"),
    (pl.col("SEXN") == 1).sum().alias("女"),
    pl.concat_list(
        pl.col("Fare").quantile(0.25),
        pl.col("Fare").quantile(0.5),
        pl.col("Fare").quantile(0.75),
    ).alias("船票费用 (25%, 50%, 75%)"),
)

状态,舱位,男,女,"船票费用 (25%, 50%, 75%)"
str,i64,u32,u32,list[f64]
"""Survived Child""",1,18,22,"[146.54, 212.93, 298.12]"
"""Survived Adult""",1,68,63,"[126.1, 243.04, 345.31]"
"""Survived Child""",3,17,17,"[100.38, 341.15, 415.16]"
"""Died""",2,94,74,"[133.07, 262.98, 385.6]"
"""Survived Child""",2,23,19,"[187.65, 327.63, 413.99]"
"""Survived Adult""",3,66,63,"[130.47, 226.86, 335.88]"
"""Died""",1,96,88,"[92.42, 220.68, 341.94]"
"""Died""",3,74,82,"[162.82, 275.15, 371.6]"
"""Survived Adult""",2,71,45,"[118.05, 257.79, 340.97]"


## <a id="PolarsUtils">4. 工具函数</a>


### 4.1 `pl.DataType` 与 `str` 的互转

`pl.DataType` 由于是 Rust 内的数据结构，如果我们需要一个 Python 对应的序列化/反序列化方式，需要将其转成字符串处理


In [47]:
from polars.datatypes import convert

int64_str = convert.dtype_to_ffiname(pl.Int64)
int64_str

'i64'

In [48]:
int64_dtype = convert.dtype_short_repr_to_dtype(int64_str)
int64_dtype == pl.Int64

True

### 4.2 使用 `Arrow` 的 C 指针

`Polars` 的底层（向量）是通过 Apache Arrow 进行存储的，如果需要访问指针级别的数据可以通过内部接口 `_export_arrow_to_c` / `_import_arrow_from_c` 实现


In [49]:
from pyext import simple

allocator = simple.SimpleAllocator([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
address = allocator.address_of()
simple.test(*address)

Export ok? 1
1
2
3
4
5
6
7
8
9
10


In [50]:
pl.Series("x", [11, 22])._export_arrow_to_c(*address)
simple.test(*address)

11
22


### 4.3 `shift`

`shift` 是 `polars` 中的一个特殊函数，用于将列向上/向下移动，类似于 `pandas` 中的 `shift`，这个函数非常有用，可以用于计算差分，或者计算滞后值等。


In [51]:
pl.DataFrame({"x": [1, 2, 3], "y": [4, 5, 6]}).with_columns(
    pl.col("x").shift(-1).alias("x_shifted")
)

x,y,x_shifted
i64,i64,i64
1,4,2
2,5,3
3,6,null


### 4.4 None，Null 和 NaN

polars 中的 `None` 和 `Null` 是一样的，都是代表空值，而 `NaN` 是一个真的值，不会影响 `has_nulls` 的结果。`None`/`Null` 不能参与运算。


In [52]:
df = pl.DataFrame(
    {"x": [1.1, np.nan, 2.2], "y": [1, None, 2]},
)
df

x,y
f64,i64
1.1,1
NaN,null
2.2,2


In [53]:
df.select(
    (pl.col("x").cast(pl.String) + " is " + pl.col("x").is_null()).alias("x_is_null"),
    (pl.col("x").cast(pl.String) + " is " + pl.col("x").is_nan()).alias("x_is_nan"),
    (pl.col("y").cast(pl.String) + " is " + pl.col("y").is_null()).alias("y_is_null"),
    (pl.col("y").cast(pl.String) + " is " + pl.col("y").is_nan()).alias("y_is_nan"),
)

x_is_null,x_is_nan,y_is_null,y_is_nan
str,str,str,str
"""1.1 is false""","""1.1 is false""","""1 is false""","""1 is false"""
"""NaN is false""","""NaN is true""",null,null
"""2.2 is false""","""2.2 is false""","""2 is false""","""2 is false"""


## 5. Exercises

### 5.1 习题一 (🌟)

1. 读取 Cars 数据集
2. 对奥迪 (Audi) 品牌下不同的车型 (Model) 按照年份 (Year) 进行升序排序
3. 添加一列 `Price (CNY)`，将 Price 列的值转换为人民币（按照[8 月 2 日 UTC 上午 1:31 汇率 1 美元 等于 7.24 人民币](https://g.co/kgs/cxE5tVi)）
4. 获取不同车型 (Model) 在不同能源类型 (Fuel Type) 下的人民币价格统计量，包括 `mean`:`Price.Mean (CNY)`, `std`:`Price.Std (CNY)`, `range (min~max)`:`Price.Range (CNY)`
5. 车型 (Model) 按照字符序，能源类型 (Fuel Type) 按照 ["Petrol"->"燃油", "Electric"->"纯电", "Hybrid"->"混动", "Diesel"->"柴油"] 排序

### 5.2 习题二 (🌟🌟)

1. 读取 Warfarin 数据集（正确处理 `.` 为 Null）
2. 添加 `Age Group`（Enum） 列，分为 `Young` (0~30], `Middle` (30~50], `Old` (50~)
3. 将 `SEX` 转为 `SEXN`，并添加 `SEX` (Enum) 列，`SEXN` 为 `SEX` 的数字表示，`SEX` 为 `SEXN` 的字符表示，`SEXN` 为 `0` 时，`SEX` 为 `F`，`SEXN` 为 `1` 时，`SEX` 为 `M`
4. 添加 `Weight Group`（Enum） 列，对于男性(SEXN=1)，分为 `Slim` (0~60], `Normal` (60~80], `Heavy` (80~), 对于女性(SEXN=0)，分为 `Slim` (0~50], `Normal` (50~70], `Heavy` (70~)
5. 获取每个 `Age Group` 中子分组的数量，包含以下信息: `Male`, `Female`, `Slim`, `Normal`, `Heavy`, `Total` 并移除多余的列。最终结果按照 `Age Group` 升序排序

### 5.3 习题三 (🌟🌟🌟)

1. 读取 Warfarin 数据集（正确处理 `.` 为 Null）
2. 移除 `DVID` = 2 的行后移除 `DVID` 列
3. 补充一列 `NEW.IND`，表示这一行开始是一个新的 `ID`，1 表示 True，0 表示 False
4. 以 `NEW.IND` 为基准，将数据集中对于的协变量信息移除 (`WT`, `AGE`, `SEX`, `DOSE`)，全部设为 None
5. 为 `ID` 为 3 的倍数的受试者添加观测点，观测点为 TIME=0 ～ 120 的 1000 个等间距点，有序插入至原有的时间点，需要补充的列信息为 MDV=1，EVID=0，其余为 None
